<a href="https://colab.research.google.com/github/FMurray/hyperdemocracy/blob/main/hyper_democracy_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
%%capture
!pip install datasets
!pip install langchain==0.0.193
!pip install chromadb
!pip install openai
!pip install tiktoken
!pip install rich

In [ ]:
import os
import random
import re

from datasets import load_dataset
import rich

# Setup Keys

In [ ]:
# os.environ["OPENAI_API_KEY"] = "YOUR KEY HERE"
import getpass
key = getpass.getpass()

In [ ]:
os.environ['OPENAI_API_KEY'] = key

# Load Demo Records

We are going to use a small subset of records provided by https://assembled.app/.

For the purposes of this workshop, we have created a huggingface dataset https://huggingface.co/datasets/hacdc/hyperdemocracy which we can load using the `load_dataset` function to returns a Huggingface dataset. See more info here [datasets](https://huggingface.co/docs/datasets/index) package.

In [ ]:
ds = load_dataset("hacdc/hyperdemocracy", split="train")

In [ ]:
ds

# Convert to Pandas DataFrame

We could do all the processing using the HF dataset, but sometimes it is simpler to explore with a DataFrame.

In [ ]:
df_all = ds.to_pandas()

In [ ]:
df_all.shape

In [ ]:
df_all.head()

# Sponsor Graph Sidequest

[Notebook here:](https://github.com/FMurray/hyperdemocracy/sidequests/sponsor_graph.ipynb)

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()
for _, record in df_all.iterrows():
    node = (record['key'], {"kind": "record", "name": record["name"]})
    G.add_nodes_from([node])
    for sponsor in record['sponsors']:
        node = (sponsor[0], {"name_tag": sponsor[1], "kind": "person"})
        G.add_nodes_from([node])
        edge = (record['key'], sponsor[0], {"kind": sponsor[2]})
        G.add_edges_from([edge])

In [ ]:
for node in list(G.nodes.data())[0:5]:
    print(node)

In [ ]:
for edge in list(G.edges)[0:5]:
    print(G.edges[edge], G.nodes[edge[0]], G.nodes[edge[1]])

In [ ]:
nx.draw(G)

# Remove Records with no Text

In [ ]:
df = df_all[df_all['body']!='']
df.shape

In [ ]:
def split_key(key):
    """
    TODO: add a link explaining this notation and variable names
    """
    congress_num, legis_class, legis_num = re.match("(\d+)(\D+)(\d+)", key).groups()
    return congress_num, legis_class, legis_num

In [ ]:
def url_from_key(key):
    """Return congress.gov url from key."""
    url_map = {
        "HR": "house-bill",
        "HCONRES": "house-concurrent-resolution",
        "HRES": "house-resolution",
        "HJRES": "house-joint-resolution",
        "S": "senate-bill",
        "SCONRES": "senate-concurrent-resolution",
        "SRES": "senate-resolution",
        "SJRES": "senate-joint-resolution",
    }
    congress_num, legis_class, legis_num = split_key(key)
    url_legis_class = url_map[legis_class]
    url = f"https://www.congress.gov/bill/{congress_num}th-congress/{url_legis_class}/{legis_num}"
    return url

# From Pandas Dataframe to Langchain Document

TODO: Document me!

TODO: Try loading directly from source with langchain HTML Document loader

Langchain makes [UnstructuredText](https://unstructured-io.github.io/unstructured/examples.html)

In [ ]:
from bs4 import BeautifulSoup
from langchain.schema import Document 

In [ ]:
docs = []
for irow, row in df.iterrows():
    congress_num, legis_class, legis_num = split_key(row['key'])
    doc = Document(
        page_content=BeautifulSoup(row['body']).get_text(),
        metadata={
            'key': row['key'],
            'congress_num': congress_num,
            'legis_class': legis_class,
            'legis_num': legis_num,
            'name': row['name'],
            'summary': BeautifulSoup(row['summary']).get_text(),
            #'summary': row['summary'],
            'source': url_from_key(row['key']),

            # Note: chroma can only filter on float, str, or int
            # https://docs.trychroma.com/usage-guide#using-where-filters

            'sponsor': row['sponsors'][0][0],

            # TODO: figure out how to break theme list up in a better way
            'theme0': row['themes'][0] if row['themes'].size > 0 else ""
        },
    )
    docs.append(doc)

In [ ]:
doc

In [ ]:
print(doc.page_content)

In [ ]:
print(len(docs))

In [ ]:
doc.metadata

# Document QA Quickstart

https://python.langchain.com/en/latest/modules/indexes/getting_started.html

TODO: What is DocumentQA? 

In [ ]:
import langchain
langchain.verbose = False

# Introducing indexes

TODO: What are indexes?

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator().from_documents(docs)

In [ ]:
index

In [ ]:
query = "What are the primary themes around energy policy?"

Try some other questions for yourself!

In [ ]:
out = index.query(query)
out

In [ ]:
out = index.query_with_sources(query)

In [ ]:
out

In [ ]:
print(out['sources'])

In [ ]:
query = "Describe in 100 words the proposed solutions to climate change?"
out = index.query_with_sources(query)
out

# Step by step explanation of the DocumentQA

## Langchain Text Splitters

https://python.langchain.com/en/latest/modules/indexes/text_splitters.html

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print("The length before splitting: ", len(docs))
print("The length after splitting: ", len(split_docs))

TODO drill in on chunk size and chunk overlap

In [ ]:
split_docs[0]

## Embed and Index Doc Chunks

### Intro to embeddings

TODO: Gabe to do intro to embeddings

In [ ]:
import openai

In [ ]:
oai_embd = openai.Embedding.create(input=docs[0].page_content, model='text-embedding-ada-002')['data'][0]['embedding']
len(oai_embd), oai_embd[0:10]

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
lc_embd = embeddings.embed_documents([docs[0].page_content])
len(lc_embd[0]), lc_embd[0][0:10]

## Index Embeddings in a Vector Database

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
db = Chroma.from_documents(split_docs, embeddings)

In [ ]:
db

In [ ]:
# TODO Forrest can implement vector search in numpy and explain what it's doing

# we can just do a search of the vectors (text embeddings)
# and some vectorscores allow us to do different types of search
# https://python.langchain.com/en/latest/modules/indexes/vectorstores.html

In [ ]:
# explnain similarity types, cosine, inner-product, squared L2, 
# looks like chroma uses hnswlib which supports 3 distances (default cosine) [TODO confirm default]
# https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/chroma.py
# https://docs.trychroma.com/usage-guide#changing-the-distance-function
# https://github.com/nmslib/hnswlib/tree/master#supported-distances

# in addition langchain offers maximal marginal relevance on top of cosine
# https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/utils.py#L10

ret_docs = db.similarity_search_with_score(
    "nuclear power", 
    k=10, 
    filter={"source": "https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17"},
)

for doc in ret_docs:
    print(doc)

In [ ]:
# show that this is all the docs from filter
len([d for d in split_docs if d.metadata['source']=='https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17'])

# What are retrievers?

TODO: TL;DR 

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import OpenAI

In [ ]:
retriever = db.as_retriever(search_kwargs={'k':10})

In [ ]:
retriever

Compare the chains in the original DocumentQA quickstart with the chains here

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

# Questions

* what are the components of the RetrievalQA chain?
* what is the QA prompt?
* how would you modify the QA prompt?
* what is the difference between the following qa chain types?,
    * stuff
    * map_reduce
    * map_rerank
    * refine

# Resources

* https://github.com/hwchase17/langchain/tree/master/langchain/chains/retrieval_qa
* https://github.com/hwchase17/langchain/tree/master/langchain/chains/question_answering

In [ ]:
rich.print(qa)

## How many ways can we print a prompt? 

In [ ]:
prompt_template = qa.combine_documents_chain.llm_chain.prompt
prompt_template

In [ ]:
print(prompt_template.template)

In [ ]:
import textwrap

In [ ]:
print('\n'.join(textwrap.wrap(prompt_template.template)))

In [ ]:
rich.print(prompt_template.format(context='[CONTEXT]', question='[QUESTION]'))

In [ ]:
answer = qa("What is the solution to climate change?")

In [ ]:
answer.keys()

In [ ]:
answer['result']

In [ ]:
qaws = RetrievalQAWithSourcesChain.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [ ]:
rich.print(qaws)

In [ ]:
pt = qaws.combine_documents_chain.llm_chain.prompt

In [ ]:
rich.print(pt.format(summaries='[SUMMARIES]', question='[QUESTION]'))

In [ ]:
answer = qaws("What is the solution to climate change?")

In [ ]:
answer.keys()

In [ ]:
answer['answer'], answer['sources']

# TODO

Try alternatives to stuff

Figure out how to pass all the options to the high level constructor. 

https://github.com/hwchase17/langchain/blob/master/langchain/indexes/vectorstore.py

In [ ]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=OpenAIEmbeddings(),
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
)


Sticking this here to decide if we want to use this in the course content

https://xml.house.gov/

TODO: Sidequest on implementing a langchain document loader using this XML schema ^^^

https://www.everycrsreport.com/

# Lets make it a conversation

https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
db

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
qachat = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0), 
    db.as_retriever(), 
    memory=memory
)

In [ ]:
query = "What is the solution to climate change?"
answer = qachat(query)

In [ ]:
rich.print(answer)

In [ ]:
follow_up = "How certain is the 350 number?"
result = qachat({"question": follow_up})

In [ ]:
rich.print(result)